# Начало работы с Tensorflow

In [ ]:
%matplotlib inline
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

https://www.tensorflow.org/programmers_guide

<img src="tensors_flowing.gif"/>

# Подбираем коэффициенты квадратного уравнения: y = w1 * x^2 + w2 * x + w3

## Задаем граф выичлений

In [ ]:
# узлы в графе - тензоры, операции в графе задают связь между тензорами
# https://www.tensorflow.org/guide/graphs
graph = tf.Graph()

with graph.as_default():
    # с помощью tf.placeholder мы  
    # передаем данные из python в граф Tensorflow
    # первая позиция размерности задает число примеров в батче
    x = tf.placeholder(tf.float32, shape=[None], name='x')  # входное значение
    y = tf.placeholder(tf.float32, shape=[None], name='y')  # ожидаемый ответ
    
    # переменные позволяют сохранять значения в графе между вызовами
    # https://tensorflow.org/guide/variables
    with tf.variable_scope('coefficients') as scope:
        # создаем переменную для коэффициентов полинома
        w = tf.get_variable('w', initializer = tf.random_normal([3, 1]))
    
    # вычисляем ответ для заданного x
    with tf.name_scope('prediction') as scope:
        poly = tf.stack([x**2, x, tf.ones_like(x)], axis=1, name='poly')
        y_pred = tf.matmul(poly, w)  # y_pred = w1 * x^2 + w2 * x + w3

## Функция потерь и оптимизатор

In [ ]:
# для подбора коэффициентов полинома используем оптимизатор
optimizer = tf.train.GradientDescentOptimizer(learning_rate=3e-4)

with graph.as_default():
    # минимизируем среднеквадратичное отклонение (MSE)
    with tf.name_scope('loss') as scope:
        mse_loss = tf.losses.mean_squared_error(y, y_pred[:, 0])
        
    # оптимизатор создает операцию обновления переменных графа
    train_step = optimizer.minimize(mse_loss)

## Обучение

In [ ]:
# задаем коэффициенты
# полинома для генерации данных
POLY_COEFFS = [5, 7, 10]  

# генератор данных
def generate_data(batch_size=100):
    # y = w1 * x^2 + w2 * x + w3 + noise
    x = np.random.uniform(-10.0, 10.0, size=batch_size)
    poly = np.stack([x**2, x, np.ones_like(x)], axis=1)
    y = np.matmul(poly, POLY_COEFFS) + np.random.normal()
    return x, y

In [ ]:
loss_history = []

# граф выполняется в рамках сессии
# по завершении сесси текущие значения
# переменных перестают быть доступными
with tf.Session(graph=graph) as sess:
    # инициализируем переменные (параметры модели)
    sess.run(tf.global_variables_initializer())
    
    for i in range(2000):  # запускаем обучение
        # генерируем случайный набор данных
        # и выполняем операции на заданном семпле
        batch_x, batch_y = generate_data(batch_size=100)
        _, mse = sess.run([train_step, mse_loss],
                          feed_dict = {x: batch_x, y: batch_y})
        
        loss_history.append(mse)
        if (i + 1) % 100 == 0:
            print('MSE[%04d]: %.3f' % (i + 1, mse))

    # сравниваем подобранные веса с истинными значениями
    print('Real coeffs: %s' % POLY_COEFFS)
    print('Estimated coeffs: %s' % w.eval()[:, 0])      

## Визуализируем кривую обучения

In [ ]:
plt.plot(loss_history[50:])
plt.title('Lerning Сurve')
plt.xlabel('iteration')
plt.ylabel('mse');

## Визуализация графа вычислений

In [ ]:
## удаляем логи предыдущих запусков
# !rm -rf ./log/001/*

In [ ]:
with tf.Session(graph=graph) as sess:
    # сохраняем граф вычислений на диск
    writer = tf.summary.FileWriter('log/001', sess.graph)
    writer.close()

In [ ]:
# запускаем интерфейс визуализации
!tensorboard --port 8800 --host 0.0.0.0 --logdir ./log/001